In [4]:
import numpy as np

In [5]:
class Node():
    ''' Initializes a node with given parameters. From A3 2a.py

    Arguments:
        name: name of node (only relevant for leaves)
        left: left child (Node)
        right: right child (Node)
    '''
    def __init__(self, name, left, right):
        self.name = name
        self.left = left
        self.right = right

In [6]:
def post_order(root):
    if root != None:
        return post_order(root.left) + post_order(root.right) + [root]
    else:
        return []

In [7]:
h1 = Node("H1", None, None)
h2 = Node("H2", None, None)
h3 = Node("H3", None, None)
h4 = Node("H4", None, None)
i2 = Node("I2", h1, h2)
i3 = Node("I3", h3, h4)
i1 = Node("I1", i2, i3)

In [36]:
''' 
Input:
    seqs = list of sequences
Output:
    D = dictionary of sequences with haplotype labels as the keys (H1, H2 etc.)
'''
def sequence_dict(seqs):
    D = {}
    n = len(seqs)

    for x in range(n):
        D["H" + str(x+1)] = seqs[x]

    return D


'''
Input:
    D = dictionary of sequences
    lst = Post order traversal
Output:
    min_score_matrix = dictionary of dictionaries: minimum score matrix (4 states per internal node)
'''
def sankoff(D, lst):
    char_list = ['A', 'C', 'G', 'T']
    min_score_matrix = {}
    index = 8

    # Traverse through nodes in post order
    for node in lst:
        min_score_matrix[node.name] = {'A' : np.inf, 'C' : np.inf, 'G' : np.inf, 'T' : np.inf}

        # Check if the node is a leaf
        if node.left is None and node.right is None:
            # Set the value for its state to be 0
            char = D[node.name][index]
            min_score_matrix[node.name][char] = 0

        else: 
            # recurrence relation: s(i) = min_j[c_ij + s_l(j)] + min_k[c_ik + s_r(k)]
            for char in char_list:
                # Get min left sum if there is a left child
                min_left = 0
                if node.left is not None:
                    min_left = np.inf
                    for left_char in char_list:
                        left_sum = min_score_matrix[node.left.name][left_char]
                        if char != left_char: left_sum += 1
                        min_left = min(min_left, left_sum)

                # Get min right sum if there is a right child
                min_right = 0
                if node.right is not None:
                    min_right = np.inf
                    for right_char in char_list:
                        right_sum = min_score_matrix[node.right.name][right_char]
                        if char != right_char: right_sum += 1
                        min_right = min(min_right, right_sum)

                min_score_matrix[node.name][char] = min_left + min_right

    return min_score_matrix

'''
Input:
    lst = Node: root of the tree
    min_score_matrix = dictionary of dictionaries: minimum score matrix (4 states per internal node)

Output: 
    state_matrix = dictionary: minimum score matrix (1 key per internal node)
'''
def backtrace(min_score_matrix, root):
    state_matrix = {}
    char_list = ['A', 'C', 'G', 'T']

    state = None
    min_score = np.inf
    score_dict = min_score_matrix[root.name]

    # Get state with minimum parismony score for the root node
    for char in score_dict:
        curr_score = score_dict[char]
        if curr_score < min_score:
            min_score = curr_score
            state = char
        
    state_matrix[root.name] = state
    
    # Backtrace on the rest of the tree
    def recursive_call(node, score, char):
        # Loop through pairs of characters for children
        for left_char in char_list:
            for right_char in char_list:
                left_score = 0
                left_cost = 0
                right_score = 0
                right_cost = 0

                # Get the score for the children 
                if node.left is not None:
                    left_score = min_score_matrix[node.left.name][left_char]
                    if left_char != char: left_cost = 1
                if node.right is not None:
                    right_score = min_score_matrix[node.right.name][right_char]
                    if right_char != char: right_cost = 1

                # Check that the sum of the children's score equals this nodes min score
                if left_score + left_cost + right_score + right_cost == score:
                    # Set scores for children and recurse
                    if node.left is not None:
                        state_matrix[node.left.name] = left_char
                        recursive_call(node.left, left_score, left_char)

                    if node.right is not None:
                        state_matrix[node.right.name] = right_char
                        recursive_call(node.right, right_score, right_char)
         

    recursive_call(root, min_score, state)

    return state_matrix

In [37]:
root = i1
# test with a3 data
sequences = ["ATGAAGGGTAGGTTCTGTGGGTGA", "ATGAAGGGAAGGTTCTGTGGGTGA", "ATGAGGGGTAGGTTCTGTGGGTGA", "ATGAGGGGTAGGTTCTGTGGGTGA"]
D = sequence_dict(sequences)
print(D)

{'H1': 'ATGAAGGGTAGGTTCTGTGGGTGA', 'H2': 'ATGAAGGGAAGGTTCTGTGGGTGA', 'H3': 'ATGAGGGGTAGGTTCTGTGGGTGA', 'H4': 'ATGAGGGGTAGGTTCTGTGGGTGA'}


In [38]:
# Get root node and traverse tree in post order
post_order_lst = post_order(root)

for node in post_order_lst:
    print(node.name)

H1
H2
I2
H3
H4
I3
I1


In [39]:
# Run sankoff's algorithm to get the state for each node
min_score_matrix = sankoff(D, post_order_lst)
state_matrix = backtrace(min_score_matrix, root)

In [40]:
print(min_score_matrix)

{'H1': {'A': inf, 'C': inf, 'G': inf, 'T': 0}, 'H2': {'A': 0, 'C': inf, 'G': inf, 'T': inf}, 'I2': {'A': 1, 'C': 2, 'G': 2, 'T': 1}, 'H3': {'A': inf, 'C': inf, 'G': inf, 'T': 0}, 'H4': {'A': inf, 'C': inf, 'G': inf, 'T': 0}, 'I3': {'A': 2, 'C': 2, 'G': 2, 'T': 0}, 'I1': {'A': 2, 'C': 3, 'G': 3, 'T': 1}}


In [41]:
print(state_matrix)

{'I1': 'T', 'I2': 'T', 'H1': 'T', 'H2': 'A', 'I3': 'T', 'H3': 'T', 'H4': 'T'}
